In [1]:
import datasets
import utils
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
import pandas as pd
import pickle_utils as pu
import imp
import GPy
import gmm_impute

In [101]:
dset_complete = datasets.datasets()["BostonHousing"][0].astype(np.float64)
dset = pu.load("impute_benchmark/amputed_BostonHousing_MCAR_rows_0.1.pkl.gz")
mog = pu.load("impute_benchmark/imputed_BGMM_20_BostonHousing_MCAR_rows_0.1/params.pkl.gz")
mog['means'] = mog['means'][:, 1:]
mog['covariances'] = mog['covariances'][:, 1:, 1:]

test_mask = np.random.rand(len(dset)) < 0.2

norm_mean = dset[~test_mask].mean()
norm_std = dset[~test_mask].std()

dset = (dset - norm_mean) / norm_std
dset_nan = dset.copy()
dset = dset.fillna(0)
dset_var = dset.applymap(lambda x: 1 if np.isnan(x) else 0)

y_train = dset.values[~test_mask, 2]
X_train = np.concatenate([dset.values[~test_mask, :2], dset.values[~test_mask, 3:]], axis=1)
X_train_var = np.concatenate([dset_var.values[~test_mask, :2], dset_var.values[~test_mask, 3:]], axis=1)
X_train_nan = np.concatenate([dset_var.values[~test_mask, :2], dset_nan.values[~test_mask, 3:]], axis=1)
y_test = dset.values[test_mask, 2]
X_test = np.concatenate([dset_var.values[test_mask, :2], dset.values[test_mask, 3:]], axis=1)
X_test_var = np.concatenate([dset_var.values[test_mask, :2], dset_var.values[test_mask, 3:]], axis=1)
X_test_nan = np.concatenate([dset_var.values[test_mask, :2], dset_nan.values[test_mask, 3:]], axis=1)

dset_complete_norm = ((dset_complete - norm_mean)/norm_std)
X_train_complete = np.concatenate([dset_complete_norm.values[~test_mask, :2],
                                   dset_complete_norm.values[~test_mask, 3:]], axis=1)
X_test_complete = np.concatenate([dset_complete_norm.values[test_mask, :2],
                                   dset_complete_norm.values[test_mask, 3:]], axis=1)




+++ Importing BostonHousing
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Servo
+++ Importing Soybean
+++ Importing LetterRecognition
+++ Importing BreastCancer
+++ Importing Shuttle


### Uncertain-Kernel underestimation of similarity.
Watch the RBF kernel matrix for these 4 points, and the Uncertain-RBF kernel matrix for the same points below. Even when just doing mean imputation the kernel distance is underestimated less, which results in better performance on the overall imputation task.

In [3]:
dset_complete_v = (dset_complete.iloc[[0,1,2,8]] - norm_mean) / norm_std
dset_complete_v

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
1,-0.450075,0.239686,-1.249448,0.32063,-0.119632,0.420212,-0.068943,0.106244,-0.970800,-0.634514,-1.456772,0.440488,-1.043539
2,-0.447416,-0.515593,-0.557479,0.32063,-0.696058,0.203324,0.410025,0.517378,-0.853727,-0.964166,-0.290468,0.440488,-0.472334
3,-0.447418,-0.515593,-0.557479,0.32063,-0.696058,1.279313,-0.212284,0.517378,-0.853727,-0.964166,-0.290468,0.394368,-1.173982
9,-0.424115,0.008906,-0.441182,0.32063,-0.236588,-0.909282,1.147706,1.040026,-0.502508,-0.542944,-1.503424,0.324110,2.382314


In [4]:
k = GPy.kern.RBF(dset_complete.shape[1])
print(k.K(dset_complete_v.values).round(3))

[[ 1.     0.162  0.149  0.   ]
 [ 0.162  1.     0.361  0.002]
 [ 0.149  0.361  1.     0.   ]
 [ 0.     0.002  0.     1.   ]]


In [5]:
dcmat = ((dset_complete - norm_mean)/norm_std).values

diff = dset_nan.values[:, np.newaxis, :] - dcmat[np.newaxis, :, :]
missing = np.isnan(diff)
diff[missing] = 0
dist = np.sum(diff**2, axis=2)
correct = missing.shape[2] / np.sum(~missing, axis=2)
bad_kernel_dist = np.exp(-.5 * dist * correct)

knn = np.argsort(-bad_kernel_dist, axis=1)[:, 1:6]
m = dcmat[knn]

diff = m[:, np.newaxis, :, np.newaxis, :] - m[np.newaxis, :, np.newaxis, :, :]
all_dist = np.sum(diff**2, axis=4)
dist = np.mean(all_dist, axis=(2,3))
#K = np.mean(all_K, axis=(2,3))
K = np.exp(-.5 * dist)
K.shape

(506, 506)

In [6]:
K.flat[::K.shape[0]+1] = 1
K[:4, :4].round(3)

array([[ 1.   ,  0.037,  0.117,  0.09 ],
       [ 0.037,  1.   ,  0.046,  0.035],
       [ 0.117,  0.046,  1.   ,  0.364],
       [ 0.09 ,  0.035,  0.364,  1.   ]])

In [7]:
dset_nan_v = dset_nan.iloc[[0,1,2,8], :]
dset_nan_v

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
1,NaN,NaN,-1.249448,0.320630,-0.119632,NaN,-0.068943,NaN,-0.970800,-0.634514,-1.456772,0.440488,-1.043539
2,-0.447416,NaN,-0.557479,-3.110981,NaN,NaN,0.410025,NaN,-0.853727,-0.964166,NaN,NaN,-0.472334
3,-0.447418,-0.515593,-0.557479,0.320630,-0.696058,1.279313,-0.212284,0.517378,-0.853727,-0.964166,-0.290468,0.394368,-1.173982
9,-0.424115,0.008906,-0.441182,0.320630,NaN,NaN,NaN,1.040026,-0.502508,-0.542944,-1.503424,NaN,2.382314


In [8]:
#full_mog = pu.load("impute_benchmark/imputed_BGMM_20_BostonHousing_MCAR_rows_0.1/params.pkl.gz")
#uk = mog_rbf.UncertainMoGRBFWhite(dset_nan_v.shape[1], full_mog)
#print(uk.K(dset_nan_v.values).round(3))

In [9]:
print(k.K(dset_nan_v.fillna(0).values).round(3))

[[ 1.     0.     0.095  0.001]
 [ 0.     1.     0.     0.   ]
 [ 0.095  0.     1.     0.   ]
 [ 0.001  0.     0.     1.   ]]


### Large variance in density estimation
Below are shown the square root of the covariance matrices of the density distribution for points "1" and "2". Notice the large standard deviations in the diagonal!

### Remedy: k-Nearest Neighbours

We can fill the values of a point with the values of the $k$ neighbours closest to it, and calculate the estimated expected kernel from that.

In [10]:
def adapted_dist(a, b):
    diff = a-b
    mask = ~np.isnan(diff)
    return np.dot(diff[mask], diff[mask]) * len(mask) / np.sum(mask)

def dist_matrix(df):
    out = np.zeros([len(df)]*2)
    for i in range(len(df)-1):
        for j in range(i+1, len(df)):
            out[j, i] = adapted_dist(df.values[j], df.values[i])
    return out + out.T
#d = 1.949611
M = dist_matrix(dset_nan_v)
#adapted_dist(*dset_nan_v.values[[0, 2]])
k.K(dset_complete_v.values).round(3), np.exp(-.5*M).round(3),

(array([[ 1.   ,  0.162,  0.149,  0.   ],
        [ 0.162,  1.   ,  0.361,  0.002],
        [ 0.149,  0.361,  1.   ,  0.   ],
        [ 0.   ,  0.002,  0.   ,  1.   ]]),
 array([[ 1.   ,  0.   ,  0.185,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ],
        [ 0.185,  0.   ,  1.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  1.   ]]))

In [11]:
class CorrectedRBF(GPy.kern.src.kern.Kern):
    def __init__(self, input_dim, white_var=1., rbf_var=1., lengthscale=None, ARD=False, active_dims=None, name='corrected_rbf'):
        super(CorrectedRBF, self).__init__(input_dim, active_dims, name)
        #self.white_var = GPy.core.parameterization.Param('white_var', white_var)
        self.rbf_var = GPy.core.parameterization.Param('rbf_var', rbf_var)
        if lengthscale is None:
            if ARD:
                lengthscale = np.ones([input_dim], dtype=np.float64)
            else:
                lengthscale = np.ones([], dtype=np.float64)
        self.lengthscale = GPy.core.parameterization.Param('lengthscale', lengthscale)
        #self.link_parameters(self.white_var, self.rbf_var, self.lengthscale)
        self.link_parameters(self.rbf_var, self.lengthscale)

        
    def K(self, X, X2=None):
        if X2 is None:
            V = X
        else:
            V = X2
        diff = X[:, np.newaxis, :] - V[np.newaxis, :, :]
        missing = np.isnan(diff)
        diff[missing] = 0
        correct = missing.shape[2] / np.sum(~missing, axis=2)
        out = self.rbf_var * np.exp(-.5 * np.sum(diff**2/self.lengthscale, axis=2) * correct)
        if X2 is None:
            out.flat[::out.shape[0]] += self.white_var
        return out

    def Kdiag(self, X):
        return (self.rbf_var)*np.ones([len(X)], dtype=np.float64)
    
    def update_gradients_full(self, dL_dK, X, X2):
        pass

In [12]:
from paramz.caching import Cache_this

class KNNRBF(CorrectedRBF):
    def __init__(self, input_dim, complete_dset, n_neighbours=5, white_var=1., rbf_var=1., lengthscale=None, ARD=False, active_dims=None, name='corrected_rbf'):
        super(KNNRBF, self).__init__(input_dim, white_var, rbf_var, lengthscale, ARD, active_dims, name)
        self.n_neighbours = n_neighbours
        self.complete_dset = complete_dset

    #@Cache_this(limit=3, ignore_args=())
    def neighbours(self, X):
        diff = X[:, np.newaxis, :] - self.complete_dset[np.newaxis, :, :]
        missing = np.isnan(diff)
        diff[missing] = 0
        dist = np.sum((diff/self.lengthscale)**2, axis=2)
        correct = missing.shape[2] / np.sum(~missing, axis=2)
        bad_kernel_dist = np.exp(-.5 * dist * correct)
        neighbours_i = np.argsort(bad_kernel_dist, axis=-1)[..., -1:] #-self.n_neighbours:-1]
        return self.complete_dset[neighbours_i]

    #@Cache_this(limit=3, ignore_args=())
    def _K_components(self, X, X2):
        #X_samples = self.neighbours(X).copy()
        #observed = ~np.isnan(X)
        #np.transpose(X_samples, (1, 0, 2))[:, observed] = X[observed]
        X_samples = X[:, np.newaxis, :]
        if X2 is None:
            V_samples = X_samples
        else:
            #V_samples = self.neighbours(X2).copy()
            #V_observed = ~np.isnan(X2)
            #np.transpose(V_samples, (1, 0, 2))[:, V_observed] = X2[V_observed]
            V_samples = X2[:, np.newaxis, :]

        diff = X_samples[:, np.newaxis, :, np.newaxis, :] - V_samples[np.newaxis, :, np.newaxis, :, :]
        diff_sq = (diff/self.lengthscale)**2
        all_rad = np.sum(diff_sq, axis=4)
        kernel = np.exp(-.5 * all_rad)
        return kernel, diff, diff_sq, all_rad
    
    #@Cache_this(limit=3, ignore_args=())
    def K(self, X, X2=None):
        kernel, _, _, _ = self._K_components(X, X2)
        out = self.rbf_var * np.mean(kernel, axis=(2,3))
        #if X2 is None:
        #    out.flat[::out.shape[0]+1] = self.rbf_var  # + self.white_var
        return out
    
    #@Cache_this(limit=3, ignore_args=())
    def update_gradients_full(self, dL_dK, X, X2):
        #if X2 is None:
        #    self.white_var.gradient = np.trace(dL_dK)
        #else:
        #    self.white_var.gradient = 0.0

        kernel, diff, diff_sq, all_rad = self._K_components(X, X2)
        prob_norm = kernel.shape[2] * kernel.shape[3]
        self.rbf_var.gradient = np.sum(dL_dK * np.mean(kernel, axis=(2,3)))
        
        if self.lengthscale.shape[0] == 1:
            grad_l = dL_dK * np.mean(all_rad * kernel, axis=(2,3)) / self.lengthscale
        else:
            grad_l = dL_dK[:, :, np.newaxis] * np.mean(
                diff**2 * kernel[:, :, :, :, np.newaxis], axis=(2,3)) / self.lengthscale**3
        grad_l = np.sum(grad_l, axis=(0, 1)) * self.rbf_var
        self.lengthscale.gradient = grad_l
        print("update_Gradients_full")
        print(self.lengthscale.gradient, self.rbf_var.gradient)
        print(np.array(self.lengthscale), np.array(self.rbf_var))
        
    #@Cache_this(limit=3, ignore_args=())
    def update_gradients_diag(self, dL_dKdiag, X):
        """
        Given the derivative of the objective with respect to the diagonal of
        the covariance matrix, compute the derivative wrt the parameters of
        this kernel and stor in the <parameter>.gradient field.
        See also update_gradients_full
        """
        g = np.sum(dL_dKdiag)
        self.rbf_var.gradient = g
        self.white_var.gradient = g
        self.lengthscale.gradient = 0.0

In [145]:
import knn_kernel
imp.reload(knn_kernel)
from knn_kernel import RBFWhiteKNNCheating

In [119]:
#_lengthscale = np.array(m.kern.lengthscale)
#_white_var = np.array(m.kern.white_var)
#_rbf_var = np.array(m.kern.rbf_var)
_lengthscale = 1.3829171
_rbf_var =  0.85418
_white_var = 0.1

In [135]:
rbfk = GPy.kern.RBF(X_train.shape[1], variance=_rbf_var,
                    lengthscale=_lengthscale, ARD=False) + GPy.kern.White(X_train.shape[1], variance=_white_var)
print(rbfk.K(X_train_complete[:4]))

rbfk.update_gradients_full(rbfk.K(X_train_complete), X_train_complete, None)
rbfk.white.variance.gradient, rbfk.rbf.variance.gradient, rbfk.rbf.lengthscale.gradient

[[ 0.95418     0.37180111  0.35598372  0.20776747]
 [ 0.37180111  0.95418     0.49114023  0.43354751]
 [ 0.35598372  0.49114023  0.95418     0.73788622]
 [ 0.20776747  0.43354751  0.73788622  0.95418   ]]


(array([ 380.71782]), array([ 5680.69451979]), array([ 4747.04128843]))

In [141]:
pknn = RBFWhiteKNN(X_train_complete, rbf_var=_rbf_var, white_var=_white_var,
                    lengthscale=_lengthscale, ARD=False)
print(pknn.K(X_train_complete[:4]))
pknn.update_gradients_full(pknn.K(X_train_complete), X_train_complete, None)
pknn.white.variance.gradient, pknn.rbf.variance.gradient, pknn.rbf.lengthscale.gradient

[[ 0.95418     0.37180111  0.35598372  0.20776747]
 [ 0.37180111  0.95418     0.49114023  0.43354751]
 [ 0.35598372  0.49114023  0.95418     0.73788622]
 [ 0.20776747  0.43354751  0.73788622  0.95418   ]]


(array([ 380.71782]), array([ 5680.69451979]), array([ 4747.04128843]))

# Here starts the actual regressor faceoff

In [123]:
def r_squared(y_true, y):
    u = np.sum((y-y_true)**2)
    v = np.sum((y_true - np.mean(y_true))**2)
    return 1 - u/v
def rmse(y_true, y):
    return np.mean((y_true - y)**2)

In [18]:
np.all(np.isfinite(y_train)), np.all(np.isfinite(X_train)), np.all(np.isfinite(X_train_nan)), np.all(np.isfinite(y_test)), np.all(np.isfinite(X_test)), np.all(np.isfinite(X_test_nan))

(True, True, False, True, True, False)

In [143]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1])/3),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
#train_perf = rf.score(X_train, y_train)
#test_perf = rf.score(X_test, y_test)
#print("Test: {:.4f}, Training: {:.4f}".format(test_perf, train_perf))
y_train_pred = rf.predict(X_train)
print("Train:", rmse(y_train, y_train_pred))
y_test_pred = rf.predict(X_test)
print("Test:", rmse(y_test, y_test_pred))

for i in range(0):
    rf.fit(X_train, y_train - y_train_pred)
    y_train_pred += rf.predict(X_train)
    print("Train {:d}:".format(i), rmse(y_train, y_train_pred))
    y_test_pred += rf.predict(X_test)
    print("Test {:d}:".format(i), rmse(y_test, y_test_pred))


Train: 0.0044938320452
Test: 0.265921953233


In [147]:
import GPy
import time
start = time.time()
m = GPy.models.GPRegression(X_train_nan, y_train[:,np.newaxis],
                            kernel=RBFWhiteKNNCheating(X_train_complete, ARD=True)
                           )
print(m.optimize(optimizer='bfgs'))
print("Training time:", time.time() -start, "seconds")
start = time.time()

y_train_pred = m.predict(X_train_nan)[0].flatten()
print("Train:", rmse(y_train, y_train_pred))
y_test_pred = m.predict(X_test_nan)[0].flatten()
print("Test:", rmse(y_test, y_test_pred))
print("Test time:", time.time() -start, "seconds")

m.kern.print_times()

m.kern

Optimizer: 				 L-BFGS-B (Scipy implementation)
f(x_opt): 				 249.626
Number of function evaluations: 	 111
Optimization status: 			 Converged
Time elapsed: 				 0:01:41.975566

Training time: 103.68197178840637 seconds
Train: 0.0450459458188
Test: 0.177922877924
Test time: 0.3075854778289795 seconds
neighbours time: 4.78781533241272 seconds


knn.,value,constraints,priors
rbf.variance,1.25069728062,+ve,
rbf.lengthscale,"(12,)",+ve,
white.variance,0.0408907338565,+ve,


In [148]:
import GPy
import time
import openblas_num_threads as openblas

with openblas.num_threads(2):
    start = time.time()
    m = GPy.models.GPRegression(X_train_nan, y_train[:,np.newaxis],
                                kernel=RBFWhiteKNNCheating(X_train_complete, ARD=True, n_neighbours=3)
                               )
    print(m.optimize(optimizer='bfgs'))
    print("Training time:", time.time() -start, "seconds")
    start = time.time()

    y_train_pred = m.predict(X_train_nan)[0].flatten()
    print("Train:", rmse(y_train, y_train_pred))
    y_test_pred = m.predict(X_test_nan)[0].flatten()
    print("Test:", rmse(y_test, y_test_pred))
    print("Test time:", time.time() -start, "seconds")

    m.kern.print_times()

m.kern

AttributeError: /home/adria/venv/bin/python3: undefined symbol: openblas_set_num_threads

In [128]:
import GPy
m2 = GPy.models.GPRegression(X_train_complete, y_train[:,np.newaxis],
                            kernel=GPy.kern.RBF(X_train.shape[1], ARD=True) + GPy.kern.White(X_train.shape[1])
                           )
print(m2.optimize(optimizer='bfgs'))
y_train_pred = m2.predict(X_train_complete)[0].flatten()
print("Train:", rmse(y_train, y_train_pred))
y_test_pred = m2.predict(X_test_complete)[0].flatten()
print("Test:", rmse(y_test, y_test_pred))

Optimizer: 				 L-BFGS-B (Scipy implementation)
f(x_opt): 				 219.657
Number of function evaluations: 	 113
Optimization status: 			 Converged
Time elapsed: 				 0:00:03.751702

Train: 0.0916575548475
Test: 0.15630385646


In [77]:
m2.kern

knn.,value,constraints,priors
sum.rbf.variance,0.734089641845,+ve,
sum.rbf.lengthscale,"(12,)",+ve,
sum.white.variance,1.0,+ve,


In [63]:
import GPy
m2 = GPy.models.GPRegression(X_train_complete, y_train[:,np.newaxis],
                            kernel=GPy.kern.RBF(X_train.shape[1], ARD=True) + GPy.kern.White(X_train.shape[1])
                           )
print(m2.optimize(optimizer='bfgs'))
y_train_pred = m2.predict(X_train_complete)[0].flatten()
print("Train:", rmse(y_train, y_train_pred))
y_test_pred = m2.predict(X_test_complete)[0].flatten()
print("Test:", rmse(y_test, y_test_pred))
m2.kern

Optimizer: 				 L-BFGS-B (Scipy implementation)
f(x_opt): 				 235.030
Number of function evaluations: 	 146
Optimization status: 			 Converged
Time elapsed: 				 0:00:05.112302

Train: 0.0984362175666
Test: 0.136451488985


sum.,value,constraints,priors
rbf.variance,0.945813971104,+ve,
rbf.lengthscale,"(12,)",+ve,
white.variance,0.0592193822493,+ve,


In [23]:
m2.kern.lengthscale

index,GP_regression.rbf.lengthscale,constraints,priors
[0],10.32058803,+ve,
[1],0.58965222,+ve,
[2],1.00000000,+ve,
[3],0.50366224,+ve,
[4],98.05990293,+ve,
[5],8.66720458,+ve,
[6],141.49786358,+ve,
[7],0.34257290,+ve,
[8],0.87194708,+ve,
[9],3.91857484,+ve,


In [39]:
optimal_lengthscale = np.array(m.kern.lengthscale)

In [351]:
# Ground-truth RBF kernel
m.kern.K(X_train_complete)

array([[ 0.82875605,  0.526138  ,  0.5119024 , ...,  0.28232418,
         0.24598097,  0.225607  ],
       [ 0.526138  ,  0.82875605,  0.62199017, ...,  0.43940945,
         0.43584892,  0.42275461],
       [ 0.5119024 ,  0.62199017,  0.82875605, ...,  0.24261259,
         0.37036444,  0.28296319],
       ..., 
       [ 0.28232418,  0.43940945,  0.24261259, ...,  0.82875605,
         0.57008071,  0.62453154],
       [ 0.24598097,  0.43584892,  0.37036444, ...,  0.57008071,
         0.82875605,  0.76806324],
       [ 0.225607  ,  0.42275461,  0.28296319, ...,  0.62453154,
         0.76806324,  0.82875605]])

In [335]:
import GPy
m = GPy.models.SparseGPRegression(X_train, y_train[:,np.newaxis],
                            kernel=GPy.kern.Matern32(X_train.shape[1], ARD=False) + GPy.kern.White(X_train.shape[1]),
                                  num_inducing=20,
                                  X_variance=X_train_var + 1e-6
                            )
m.optimize()
y, _ = m.predict(X_train)
print("Train:", rmse(y_train, y.flatten()))
y, _ = m.predict(X_test)
print("Test:", rmse(y_test, y.flatten()))

KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [104]:
imp.reload(gmm_impute)
gmm_impute._gmm_impute(mog, X_train_nan[9:10], 1)

(array([[[-0.42006788,  1.01905536,  0.68299323, -0.40612554,  0.63205938,
          -0.5832783 ,  0.39105928,  0.67011543,  0.35313276, -0.21148562,
           0.3739001 ,  1.10710231]]]),
 array([[ 0.63855337,  0.32716695,  0.        ,  1.04207718, -0.0528659 ,
          1.04048448,  0.61910684,  0.2997959 ,  0.54710433,  1.24750701,
          0.30055021,  0.        ]]))

In [21]:
m.kern

sum.,value,constraints,priors
rbf.variance,0.86458246011,+ve,
rbf.lengthscale,1.41275603781,+ve,
white.variance,0.00589876098757,+ve,


In [109]:
imp.reload(mog_rbf)
d = mog_rbf.uncertain_point(X_train_nan[9], mog, 1., np.eye(12), single_gaussian_moment_matching=True)
d['means'], d['covariances']

(array([[[-0.42006788],
         [ 1.01905536],
         [-0.40612554],
         [ 0.63205938],
         [-0.5832783 ],
         [ 0.39105928],
         [ 0.67011543],
         [ 0.35313276],
         [-0.21148562],
         [ 0.3739001 ]]]),
 array([[[ 0.63855337,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.32716695,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.04207718,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , -0.0528659 ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  1.04048448,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
  

In [66]:
imp.reload(mog_rbf)
imp.reload(gmm_impute)
uk2 = mog_rbf.UncertainMoGRBFWhite(X_train.shape[1], mog, cutoff=1.0, single_gaussian=True)
K_uk2 = uk2.K(X_train_nan, X_train_nan)

To compute map took me: 0.4738774299621582
FOR REAL [[[-0.15875905]
  [ 0.17830032]
  [-0.41094708]
  [-0.40360746]]]


In [110]:
imp.reload(mog_rbf)
imp.reload(gmm_impute)
uk3 = mog_rbf.UncertainGaussianRBFWhite(X_train.shape[1], mog)
K_uk3 = uk3.K(X_train_nan, X_train_nan)

To compute map (rectangular) took me: 0.9276993274688721
To compute matrix took me: 0.2105269432067871
(408, 408)


In [69]:
np.max(K_uk3 - K_uk3.T)

0.00090383853741499674

In [345]:
import mog_rbf
import imp
import gmm_impute
imp.reload(gmm_impute)
imp.reload(mog_rbf)

uncertain_kern = mog_rbf.UncertainMoGRBFWhite(X_train.shape[1], mog)
uncertain_k = GPy.models.GPRegression(X_train_nan, y_train[:, np.newaxis],
                                     kernel=uncertain_kern)
y, _ = uncertain_k.predict(X_train_nan)
print("Train:", rmse(y_train, y.flatten()))
y, _ = uncertain_k.predict(X_test_nan)
print("Test:", rmse(y_test, y.flatten()))

(401, 401)
Train: 0.249568009132
Test: 0.384448154724


In [135]:
import mog_rbf
import imp
import gmm_impute
imp.reload(gmm_impute)
imp.reload(mog_rbf)

uncertain_kern = mog_rbf.UncertainGaussianRBFWhite(X_train.shape[1], mog,
                                                   rbf_var=m.kern.rbf.variance,
                                                   white_var=m.kern.white.variance,
                                                   lengthscale=m.kern.rbf.lengthscale)
uncertain_k = GPy.models.GPRegression(X_train_nan, y_train[:, np.newaxis],
                                     kernel=uncertain_kern)
y, _ = uncertain_k.predict(X_train_nan)
print("Train:", rmse(y_train, y.flatten()))
y, _ = uncertain_k.predict(X_test_nan)
print("Test:", rmse(y_test, y.flatten()))

Train: 0.251347573504
Test: 0.515770105012


In [346]:
X_train_alt = X_train_nan.copy()
X_test_alt = X_test_nan.copy()

for i, X in enumerate(X_train_alt):
    mask = np.isnan(X)
    d = gmm_impute.conditional_mog(mog, X, mask)
    # Perform moment matching to get mean and variance
    # https://stats.stackexchange.com/questions/16608/what-is-the-variance-of-the-weighted-mixture-of-two-gaussians
    ms = d['means'] * d['weights'][:, np.newaxis]
    mean = np.sum(ms, axis=0)
    var_mix = np.sum(d['covariances'] * np.eye(d['covariances'].shape[1]), axis=2).sum(axis=0)
    var = var_mix + np.sum(ms * d['means'], axis=0) - mean
    X_train_alt[i, mask] = mean
    X_train_var[i, mask] = var

for i, X in enumerate(X_test_alt):
    mask = np.isnan(X)
    d = gmm_impute.conditional_mog(mog, X, mask)
    X_test_alt[i, mask] = np.sum(d['means'] * d['weights'][:, np.newaxis], axis=0)


sparse_m = GPy.models.SparseGPRegression(
    X_train_alt, y_train[:, np.newaxis],
    kernel=(GPy.kern.RBF(X_train.shape[1], ARD=False)
                   + GPy.kern.White(X_train.shape[1])),
    Z=X_train_alt,
    X_variance=X_train_var)
#sparse_m.optimize()
y, _ = sparse_m.predict(X_train_alt)
print("Train:", rmse(y_train, y.flatten()))
y, full_cov = sparse_m.predict(X_test, full_cov=True)
print("Test:", rmse(y_test, y.flatten()))

 /home/adria/venv/lib/python3.5/site-packages/paramz/transformations.py:109: RuntimeWarning:divide by zero encountered in log


Train: 0.255991811883
Test: 0.362862548471


In [271]:
full_cov

array([[  2.78635832e+00,   2.06344958e-04,   1.64838064e-05, ...,
          4.35285552e-04,   1.00000000e-15,   8.26572097e-04],
       [  2.06344958e-04,   2.94234875e+00,   1.27033141e-03, ...,
          4.72627943e-01,   1.00000000e-15,   1.87282191e-02],
       [  1.64838064e-05,   1.27033141e-03,   2.78212823e+00, ...,
          8.27680241e-04,   2.87765037e-03,   1.40391869e-02],
       ..., 
       [  4.35285552e-04,   4.72627943e-01,   8.27680241e-04, ...,
          2.93926028e+00,   1.00000000e-15,   5.84986024e-02],
       [  1.00000000e-15,   1.00000000e-15,   2.87765037e-03, ...,
          1.00000000e-15,   2.71902756e+00,   1.00000000e-15],
       [  8.26572097e-04,   1.87282191e-02,   1.40391869e-02, ...,
          5.84986024e-02,   1.00000000e-15,   2.80199682e+00]])

In [39]:
import selu
import denoising_ae as dae
import imp
imp.reload(dae)
imp.reload(selu)

tf_float = tf.float32
tf.reset_default_graph()

nn_X = tf.placeholder(tf_float, shape=[None, X_train.shape[1]], name="X")
nn_y = tf.placeholder(tf_float, shape=[None, 1], name="y")
nn_kp = tf.placeholder(tf_float, shape=[], name="keep_prob")
nn_lr = tf.placeholder(tf_float, shape=[], name="learning_rate")


nn_preds = dae.FC_net(nn_X, [(1024, selu.nlin)]*8 + [(1, None)], selu.initializer, keep_prob=nn_kp)
nn_loss = tf.reduce_mean(tf.squared_difference(nn_preds, nn_y))
nn_train = tf.train.GradientDescentOptimizer(nn_lr).minimize(nn_loss)

In [40]:
from tqdm import tqdm
import itertools

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

batch_size = 128

num_batches = int(np.ceil(X_train.shape[0] / batch_size))

pbar = tqdm(itertools.count(0))
for step in pbar:
    i = step % num_batches
    # Populate feed_dict; duplicated code for num and cat
    s = slice(batch_size * i, batch_size * (i + 1))
    feed_dict = {nn_kp: 0.95,
                 nn_lr: 0.001 if step < 800 else 0.0001,
                 nn_X: X_train[s],
                 nn_y: y_train[s, np.newaxis],
                }
    del i
    
    if step % 100 == 0:
        _, l, y = sess.run([nn_train, nn_loss, nn_preds], feed_dict)
        d = "Loss: {} ".format(l)
        d += "Train: {} ".format(r_squared(y_train[s], y.ravel()))
        y = sess.run(nn_preds, {nn_kp: 1.0,
                                nn_X: X_test,
                                nn_y: y_test[:, np.newaxis]})
        d += "Test: {} ".format(r_squared(y_test, y.ravel()))
        pbar.set_description(d)
    else:
        sess.run(nn_train, feed_dict)


0it [00:00, ?it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 1it [00:00,  3.25it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 3it [00:00,  4.23it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 5it [00:00,  5.52it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 8it [00:00,  7.19it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 11it [00:00,  9.28it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 14it [00:00, 11.47it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 17it [00:01, 13.87it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 20it [00:01, 16.15it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 23it [00:01, 18.33it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 

KeyboardInterrupt: 

In [41]:
tf.summary.FileWriter("faceoff/", graph=sess.graph)

Loss: 0.055597126483917236 Train: 0.9541863795146673 Test: 0.9235167656130201 : 3687it [02:48, 21.89it/s]

In [23]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
import time

# Fit KernelRidge with parameter selection based on 5-fold cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [RBF(l)
                         for l in np.logspace(-2, 2, 10)]}
kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid)
stime = time.time()
kr.fit(X_train[:100], y_train[:100])
print("Time for KRR fitting: %.3f" % (time.time() - stime))

Time for KRR fitting: 0.549


In [18]:
Z = np.random.rand(1000, X_train.shape[1])
rbf = GPflow.kernels.RBF(X_train.shape[1], variance=1, ARD=True)
m = GPflow.sgpr.SGPR(X_train, y_train[:,None], kern=rbf, Z=Z)
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.AdamOptimizer(), callback=logger)



[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 181586.72233875]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 180537.74127184]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 179500.65675795]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 178476.56694212]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 177466.08404523]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 176469.42592986]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 175486.5617425]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 174517.32678109]
Caught KeyboardInterrupt, setting model                  with most recent state.


In [24]:
y, _ = m.predict_y(X_train)
print("Train:", r_squared(y_train, y.flatten()))
y, _ = m.predict_y(X_test)
print("Test:", r_squared(y_test, y.flatten()))

Train: -4.42959616572
Test: -4.39652614283
